In [29]:
import json
import os
import random
from typing import List, Dict
import uuid

from openai import OpenAI, AsyncOpenAI
from pydantic import BaseModel, Extra, Field
import tiktoken

from config import settings, app_settings

from src.simschat.models import (
    CharacterSpecification,
    SceneState,
    CharacterRelationState,
    CharacterState,
    create_dynamic_enum
)

In [30]:
openai_client = OpenAI(api_key=settings.openai_api_key)
tokenizer = tiktoken.encoding_for_model("gpt-4o")

In [31]:
class Character(BaseModel):
    uid: str
    spec: CharacterSpecification
    state: CharacterState

# 0. Load Character & State

In [32]:
# Load Scene
scene_uid = "2f2462c1-bdef-4089-b206-c47decd841f3"

with open(f"simschat/scene/{scene_uid}.json", "r") as f:
    scene_dict = json.load(f)

In [33]:
scene_description = scene_dict["scene"]["revised_trope"]
scene_state = SceneState.model_validate(scene_dict["scene"]["scene"])

In [34]:
character_states = {
    scene_dict["character_states"]["character1_uid"]: CharacterState.model_validate(scene_dict["character_states"]["character1"]),
    scene_dict["character_states"]["character2_uid"]: CharacterState.model_validate(scene_dict["character_states"]["character2"]),
}

In [35]:
# Load Character
with open("simschat/character_collection.json", "r") as f:
    character_collection = json.load(f)

character_ids = list(character_collection["source"].keys())

character1_id="35f0c56f-263d-42df-846c-e1833d8ca0ab"
character2_id="00d66087-9b3b-46da-bd74-bf45cbe81d3c"

print(character1_id, character_collection["source"][character1_id])
print(character2_id, character_collection["source"][character2_id])

35f0c56f-263d-42df-846c-e1833d8ca0ab wiki_Zephyr Orion.txt
00d66087-9b3b-46da-bd74-bf45cbe81d3c wiki_Vivienne LaRoux.txt


In [36]:
# Load Characters
with open(f"simschat/characters/{character1_id}.json", "r") as f:
    character1_spec = CharacterSpecification(**json.load(f))
    
with open(f"simschat/characters/{character2_id}.json", "r") as f:
    character2_spec = CharacterSpecification(**json.load(f))

In [37]:
character1 = Character(
    uid=character1_id,
    spec=character1_spec,
    state=character_states[character1_id]
)
character2 = Character(
    uid=character2_id,
    spec=character2_spec,
    state=character_states[character2_id]
)

In [38]:
print(character1.model_dump_json(indent=4))

{
    "uid": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "spec": {
        "name": "Zephyr Orion",
        "gender": "male",
        "age": 28,
        "dialogue_tone": "playful, jovial, and engaging, with a witty humor and warmth that makes everyone feel at ease. Known for storytelling with captivating tales of space adventures.",
        "career": "Astronaut",
        "personality_traits": [
            {
                "trait": "Goofball",
                "description": "Enjoys joking and making others laugh, bringing a playful spirit to social situations."
            },
            {
                "trait": "Materialistic",
                "description": "Loves acquiring new possessions and often leans towards bragging about them."
            },
            {
                "trait": "Outgoing",
                "description": "Flourishes in social situations and enjoys being around people."
            },
            {
                "trait": "Gloomy",
                "descrip

# 1. Action Prediction

In [50]:
class CharacterAction(BaseModel):
    character: str
    think: str
    action: str
    targets: List[str]
    class Config:
        extra = Extra.forbid

class ActionsResult(BaseModel):
    actions: List[CharacterAction]
    class Config:
        extra = Extra.forbid

/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_94912/266023748.py:7: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid
/var/folders/wj/0c7skj2154q4844jqxlw3yxr0000gn/T/ipykernel_94912/266023748.py:12: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  extra = Extra.forbid


In [ ]:
ACTION_PREDICTION_INSTRUCTION = '''Given the following informations about a story scene, write a sequence of actions that the characters will perform in the scene.

# Given Informations
* Characters: 2 characters involved in the story. (identified through their uid)
* Scene Trope: the story trope that the given scene follows
* Scene Setting: location, setting of the scene

## Characters
[Character1]
{character1}

[Character2]
{character2}

## Scene Trope
{trope}

## Scene Setting
{scene}

# Action Prediction
## Definition of 'Action'
a character performs an 'action' through the following steps
* 'character': select the character that will perform this action (by uid)
* 'think': consider what the character should say and what goals the character wants to achieve. write the thinking process here
* 'action': description of the action this character will actually perform
    * the action must be detailed description of a single-verb action.
    * if there are dialogues, write the actual dialogues
    * if there are any background, items, characters involved, include the details about them
    * action must be maximum 2 sentences
    * don't make the actions obvious
* 'targets': List of characters (by uid) that will be affected. leave it empty if it there are no targets


Return a list of actions in the following json format (maximum of 3)
{{
    "actions": [
        {{
            "character": str,
            "think": str
            "action": str,
            "targets": List[str]
        }},
        ...
    ]
}}'''

In [62]:
action_pred_instruction = ACTION_PREDICTION_INSTRUCTION.format(
    character1=character1.model_dump_json(),
    character2=character2.model_dump_json(),
    trope=scene_description,
    scene=scene_state.model_dump_json()
)

In [66]:
messages = [
    {"role": "user", "content": action_pred_instruction}
]
# decode_params = {"temperature": 0.95}
decode_params = {"reasoning_effort": "low"}

response = openai_client.beta.chat.completions.parse(
    # model="gpt-4o",
    model="o3-mini",
    messages=messages,
    response_format=ActionsResult,
    **decode_params,
)
predicted_actions = response.choices[0].message.parsed.actions

In [67]:
print(len(predicted_actions))

3


In [68]:
for action in predicted_actions:
    print(action.model_dump_json(indent=4))
    print('-'*30)

{
    "character": "35f0c56f-263d-42df-846c-e1833d8ca0ab",
    "think": "Zephyr wants to break the ice with his playful storytelling, showcasing his charm and poking fun at his own love for space memorabilia while gently teasing Vivienne's high-fashion persona.",
    "action": "Approaches Vivienne with a warm smile and says, 'Even the stars seem dim without a splash of high fashion tonight!' while gesturing towards the glimmering chandeliers.",
    "targets": [
        "00d66087-9b3b-46da-bd74-bf45cbe81d3c"
    ]
}
------------------------------
{
    "character": "00d66087-9b3b-46da-bd74-bf45cbe81d3c",
    "think": "Vivienne is taken aback by Zephyr’s banter, her initial icy demeanor slowly melting as she finds his wit unexpectedly charming, pushing her to engage despite her mean streak.",
    "action": "Raises an eyebrow and replies sharply, 'I must admit, your astronomical puns shine brighter than expected,' with a half-smile that softens her tone.",
    "targets": [
        "35f0c5